In [11]:
import sys
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [43]:
filepaths=!ls *.csv
filepaths=[i for i in filepaths if '@20' in i]
filepaths

['ADMM@20.csv',
 'CDAE@20.csv',
 'EASE@20.csv',
 'FFM@20.csv',
 'GRU4RecF@20.csv',
 'ItemKNN@20.csv',
 'LGCN@20.csv',
 'MultiDAE@20.csv',
 'NeuMF@20.csv',
 'Pop@20.csv',
 'RecVAE@20.csv',
 'SLIMElastic@20.csv']

In [5]:
filepaths=[i for i in filepaths if '@' in i]

In [36]:
ratios=[1,0.7,0.7,0.9,0.35]

In [37]:
dataframe_list = []

print('순위별 가중치값 입력(ex: 1 0.9 0.8 ...)')
rank_ratio = [1 for i in range(15)]
rank_len = len(rank_ratio)

print(f"앙상블 모델 개수: {len(filepaths)}")

for i in range(len(filepaths)):
    dataframe_list.append(pd.read_csv(filepaths[i]))

user_list = dataframe_list[0]['user'].unique()
dataframe_len = len(dataframe_list)

순위별 가중치값 입력(ex: 1 0.9 0.8 ...)
앙상블 모델 개수: 5


In [38]:
len(user_list)

31360

In [39]:
K=13
result = []
tbar = tqdm(user_list, desc='Ensemble')
for user in tbar:
    temp = defaultdict(float)
    for df_idx in range(dataframe_len):
        items = dataframe_list[df_idx][dataframe_list[df_idx]['user'] == user]['item'].values
        max_rank = min(len(items), rank_len)
        for rank_idx in range(max_rank):
            temp[items[rank_idx]] += rank_ratio[rank_idx] * ratios[df_idx]

    for key, _ in sorted(temp.items(), key=lambda x: x[1], reverse=True)[:K]:
        result.append((user, key))

Ensemble: 100%|█████████████████████████████████████████████| 31360/31360 [02:14<00:00, 232.47it/s]


In [44]:
filename="--".join([f"{m.split('@')[0]}_{r}" for m,r in zip(filepaths,ratios) if "@" in m])
filename

'ADMM_1--CDAE_0.7--EASE_0.7--FFM_0.9--GRU4RecF_0.35'

In [41]:
filename="Ensemble@13"

In [42]:
submission = pd.DataFrame(result, columns=['user', 'item'])
submission.to_csv(f'{filename}.csv', index=False)

In [ ]:
ratios = []
dataframe_list = []

print('순위별 가중치값 입력(ex: 1 0.9 0.8 ...)')
rank_ratio = list(map(float, sys.stdin.readline().split()))
rank_len = len(rank_ratio)

num = int(input('앙상블할 모델 결과 개수: '))

for i in range(num):
    filepath = input(f'{i+1}번째 파일 경로: ')
    ratio = float(input(f'{i+1}번째 파일 가중치(0~1 사이 실수): '))
    dataframe_list.append(pd.read_csv(filepath))
    ratios.append(ratio)

user_list = dataframe_list[0]['user'].unique()
dataframe_len = len(dataframe_list)

result = []
tbar = tqdm(user_list, desc='Ensemble')
for user in tbar:
    temp = defaultdict(float)
    for df_idx in range(dataframe_len):
        items = dataframe_list[df_idx][dataframe_list[df_idx]['user'] == user]['item'].values
        max_rank = min(len(items), rank_len)
        for rank_idx in range(max_rank):
            temp[items[rank_idx]] += rank_ratio[rank_idx] * ratios[df_idx]

    for key, _ in sorted(temp.items(), key=lambda x: x[1], reverse=True)[:10]:
        result.append((user, key))

submission = pd.DataFrame(result, columns=['user', 'item'])
submission.to_csv(f'ensemble({num})-file.csv', index=False)